# Coweeta, NC RHESSys workflow

US Forest Service WS18 is a 12.3ha, mixed hardwood-oak, control catchment that’s been undisturbed since 1927.


RHESSysWorkflows provides a series of Python tools for performing [RHESSys](https://github.com/RHESSys/RHESSys) data preparation workflows. These tools build on the workflow system defined by [EcohydroLib](https://github.com/selimnairb/EcohydroLib) and [RHESSysWorkflows](https://github.com/selimnairb/RHESSysWorkflows).

This notebook is built on RHESSys sample workflow. Not all steps are documented. Here we focus on explaining new or modifications.

Users interested in seeing step outputs, remove **output = ** from the command line.


In [ ]:
import os
import logging
import hs_utils
from rhessys_wf import *
%matplotlib inline

The `RHESSysWorkflow` class is provided as part of the `rhessys_wf` library (imported above) to streamline your interaction with the `RHESSysWorkflows` [core functions](https://github.com/selimnairb/RHESSysWorkflows).  More information about this class can be obtained by executing the `help(RHESSysWorkflow)` command.

Create an instance of the `RHESSysWorkflow` class and assign it to the variable `w`, using a custom gage, a start date of **2008-01-01**, and an end date of **2010-01-01**.  This command will create a clean directory for the given project name of **coweeta_nc** in the JupyterHub's default data directory.

In [ ]:
w = RHESSysWorkflow(project_name='coweeta_ncv1', 
                    gageid='custom',
                    start_date='2008-01-01',
                    end_date='2010-01-01'
                    )

## Retrieve Coweeta, NC data from HydroShare Resource

### Part 1 of 3

The first step to retrieve a resource from HydroShare is to create a HydroShare connection as follows:

In [ ]:
hs = hs_utils.hydroshare()

### Part 2 of 3

Dr. Laurence Lin from UNC Institute for the Environment
has prepared a HydroShare Resource for **Coweeta, NC** [(RHESSys (5.20) watershed 18, Coweeta, NC)](https://www.hydroshare.org/resource/c397f68fbc9847d6a717b0a2c2fe7671/) 

If you follow the web link, or move your mouse cursor over the above web link, you will note the web link has nas numbers and text after **https://www.hydroshare.org/resource/**. This text string is a resource id. Here the resource id is **c397f68fbc9847d6a717b0a2c2fe7671** and is needed to retrieve the resource prepared by Dr. Laurence Lin.

In [ ]:
resource_id = 'c397f68fbc9847d6a717b0a2c2fe7671'
# get ETV data bundle for this tutotial stored on HydroShare
content = hs.getResourceFromHydroShare(resource_id)

### Part 3 of 3

After the resource has finished downloading, the next step is to unzip the folder in your workspace.
We know the name of the zipped file "ws18.zip" from the above step. If you are using a different resource, you will be required to change **hs.content['ws18.zip']**, as well as **ws18** folder names below.


In [ ]:
zipfolder = w.sub_project_folder + '/RHESSys_ETV'
w.create_path(zipfolder)
zipfilepathname = hs.content['ws18.zip']
w.unzip_etv_zip_file_at_path(zipfilepathname, zipfolder)
print 'Steps Finished'

## RegisterDEM

From the HydroShare resource downloaded above, a DEM is provided for Ws18. The DEM.tif is location in the zipfolder location. If you are using a different resource, you will need to change the demfile value below. These changes are required with other data steps below. <br/><br/>
The **os.environ["GDAL_DATA"]** indicates to PYTHON where the GDAL installation is. If you have compiled GDAL (i.e. for specialized libraries) in your environment, change this value.
<br/><br/>

**RegisterDEM** will result in the DEM being copied to your project directory, and the DEM spatial resolution will determine the default spatial resolution of your project (i.e. other rasters imported to your project will by default be resampled to the match the DEM resolution and spatial reference). Also, the DEM extent will be used to determine the bounding box for the study area; a polygon of the DEM extent will be generated and saved as a shapefile in your project directory.


This step **registers the DEM.tif** stored in the resource rather than using the `get_USGSDEMForBoundingbox` command.

In [ ]:
os.environ["GDAL_DATA"] = "/opt/conda/share/gdal"
demfile = w.sub_project_folder + '/RHESSys_ETV/ws18/DEM.tif'
options = " -p " + w.sub_project_folder + " -d " + demfile + " -b \"CWT18 DEM\""
w.RegisterDEM(w.sub_project_folder, demfile)

## Register Import Gage
We will use the **RegisterGage** tool to tell EcohydroLib the coordinates of the Dead Run streamflow gage. This tool takes as input a point shapefile containing one or more points; the WGS84 lat-lon coordinates for the desired gage will be extracted from the shapefile. These coordinates will be written to the metadata store, and a new point shapefile containing a point only for the selected gage will be created in the project directory. <br/><br/>
You will need to determine the **shape field** with the **id_attribute**. You can use GIS products from GDAL, ESRI, QGIS to determine these values.

**Register streamflow gage** coordinates from a point shapefile into metadata store for a project directory, copying the shapefile into the project directory in the process

In [ ]:
gagefile = w.sub_project_folder + '/RHESSys_ETV/ws18/gage.shp'
layername = 'gage'
id_attribute = 'gage_id'
id_value = "\"Weir 18\""
w.RegisterGage(w.sub_project_folder, gagefile, layername, id_attribute, id_value)

In [ ]:
output = w.get_USGSNLCDForDEMExtent(w.sub_project_folder)
print(output)

**Register raster** into metadata store for a project directory,
copying the raster into the project directory in the process.  Raster type must
be one of GenericMetadata.RASTER_TYPES.

In [ ]:
roads_file = w.sub_project_folder + '/RHESSys_ETV/ws18/rhessys/roads.tif'
options = " -p " + w.sub_project_folder + " -t roads " + " -r " + roads_file + " --force -s near "
w.RegisterRasterOptions(options)

## Download Soil
The USDA NRCS provides the [Soil Data Mart](http://sdmdataaccess.nrcs.usda.gov/), a web services-based interface for querying and downloading high-resolution SSURGO soils data.
<br/>
<br/>
Note: Output from get_SSURGOFeaturesForBoundingbox has been piped to output variable. For debugging, uncomment print statement (i.e. remove #).

In [ ]:
output = w.get_SSURGOFeaturesForBoundingbox(w.sub_project_folder)
#print(output)

## Prepare LAI
Here the resource or user supplied **LAI** is registered.

In [ ]:
lai_file = w.sub_project_folder + '/RHESSys_ETV/ws18/lai.tif'
options = " -p " + w.sub_project_folder + " -t lai " + " -r " + lai_file + " -s bilinear --force "
w.RegisterRasterOptions(options)

## Prepare ISOHYET
Here the resource or user supplied **ISOHYET** is registered.

In [ ]:
isohyet_file = w.sub_project_folder + '/RHESSys_ETV/ws18/isohyet.tif'
options = " -p " + w.sub_project_folder + " -t isohyet " + " -r " + isohyet_file + " --force -s bilinear "
w.RegisterRasterOptions(options)

## Generate Soil Properties From SSURGO
Here the resource or user supplied **ISOHYET** is registered.

In [ ]:
output = w.GenerateSoilPropertyRastersFromSSURGO(w.sub_project_folder)
#print(output)

## Create a new GRASS GIS Location

RHESSys requires that all spatial data used to create a world file and flow table for a RHESSys model be stored in a GRASS GIS mapset. We'll start building these data in RHESSysWorkflows by creating a new GRASS location and mapset within our project directory, and importing our DEM into this mapset using the **CreateGRASSLocationFromDEM** tool. <br/>
<br/>
Note: Output from CreateGRASSLocationFromDEM has been piped to output variable. For debugging, uncomment print statement (i.e. remove #).

In [ ]:
output = w.CreateGRASSLocationFromDEM(w.sub_project_folder, '"RHESSys model for watershed 18 at Coweeta LTER"')
#print(output)

## Import RHESSys Code
To create worldfiles and flow tables RHESSysWorkflows needs a copy of RHESSys source code. RHESSysWorkflows also uses the new RHESSys ParamDB database and Python libraries to generate vegetation, soil, land use and other parameters needed by RHESSys. RHESSysWorkflows is only compatible with the pre-release version of RHESSys 5.16 and later versions of the code. At present, and for first-time users, the most reliable way to import ParamDB and RHESSys source code into your project is to download the code from GitHub using the **ImportRHESSysSource** tool. However, this tool is also capable of importing RHESSys source code stored on your computer. This allows you to import the code from a previous RHESSysWorkflows project; ParamDB is always downloaded from GitHub, even when RHESSys source code is imported from a local source.

The `ImportRHESSysSource` function downloads and compiles the latest [RHESSys](https://github.com/RHESSys/RHESSys) source code in the user defined project folder. This make take a couple minutes.   

In [ ]:
output = w.ImportRHESSysSource(w.sub_project_folder)
#print(output)

## Delineate Watershed
RHESSysWorkflows automates the process of delineating your study watershed based on the location of the streamflow gage registered in the workflow using EcohydroLib. As part of this process, many datasets needed by RHESSys will be derived from the DEM. To delineate the watershed we use the **DelineateWatershed** tool.

Here the **cell_size** parameter specifies the minimum size (in DEM cells) for subwatersheds generated by the GRASS command r.watershed.

The **area_estimate** parameter allows you to provide a guess of the area (in sq. km) of the delineated watershed. DelineateWatershed will report whether the watershed is within 20% of the area. You can view the delineated watershed in GRASS by displaying the raster map named basin. If the area or the shape of the delineated watershed differs greatly from what you expect, you may need to vary how DelineateWatershed snaps your streamflow gage onto the stream network. This is accomplished by either changing the -s (a.k.a. --streamThreshold) or stream threshold parameter and/or the -w (a.k.a. --streamWindow) parameter passed to r.findtheriver.

### Please note: If you used a different resource from HydroShare, you will need to modify these parameters.

In [ ]:
cell_size = 250
area_estimate = 0.13
output = w.DelineateWatershed(w.sub_project_folder, cell_size, area_estimate)
#print(output)

## Generate derived landcover maps

Now we generating LAI, isohyet, roads and landcover maps from your project directory into GRASS using the **ImportRasterMapIntoGRASS** tool.
### Please note: If you are using a different resource, edit these steps accordingly.

In [ ]:
output = w.ImportRasterMapIntoGRASS(w.sub_project_folder, 'lai', 'bilinear')

In [ ]:
output = w.ImportRasterMapIntoGRASS(w.sub_project_folder, 'isohyet', 'bilinear')

In [ ]:
output = w.ImportRasterMapIntoGRASS(w.sub_project_folder, 'roads', 'nearest')

In [ ]:
output = w.ImportRasterMapIntoGRASS(w.sub_project_folder, 'landcover', 'nearest')

In [ ]:
output = w.RegisterLandcoverReclassRules(w.sub_project_folder)

Once the landcover reclass rules are in place, it is very easy to generate the raster maps derived from the landcover data as well as the vegetation and land use definition files needed by RHESSys using ***GenerateLandcoverMapsOptions*** tool.

In [ ]:
options = " -p " + w.sub_project_folder + " --skipRoads"
output = w.GenerateLandcoverMapsOptions(options)
#print(output)

## Process Soil Texture Map

Since we used EcohydroLib's SSURGO tools to generate percent sand and percent clay raster maps for our watershed, we can use the GRASS add-on r.soils.texture to generate USDA soil texture classes, for which RHESSys's ParamDB contains parameters. It is also possible to use custom soil maps (refer to the Tutorial).

To generate our soil texture map in GRASS, as well as the corresponding RHESSys soil definition files, use the **GenerateSoilTextureMap** tool as follows:


In [ ]:
output = w.GenerateSoilTextureMap(w.sub_project_folder, options='')
#print(output)

## Generate Patch Map

RHESSysWorkflows provides **GeneratePatchMap**, an automated tool for creating gridded and clumped patch maps. Gridded patch maps consist of a regular grid of the same resolution and extent as the DEM; clumped maps can be created using elevation or topographic wetness index rasters. Modelers can also use custom patch maps registered via EcohydroLib's RegisterRaster tool and imported into GRASS using ImportRasterMapIntoGRASS. 

In [ ]:
patch_options = " -p " + w.sub_project_folder + " -t grid"
output = w.GeneratePatchMapOptions(patch_options)
#print(output)

## Import Climate Data

Because of the great variability of climate data formats, and the complexity of time-series workflows, we have chosen to focus development effort on RHESSysWorkflows toward making it easier to acquire and manipulate geospatial data required for building RHESSys work files and flow tables. This means that the modeler is responsible for building the climate data necessary for building RHESSys world files and performing model runs.

RHESSysWorkflows provides the **ImportClimateData** tool to import RHESSys climate data into your project.

### Please note: If you are using a different resource, edit this step accordingly.

In [ ]:
w.climate_data_fullpathname = w.sub_project_folder + '/RHESSys_ETV/ws18/rhessys/clim'
output = w.ImportClimateData(w.sub_project_folder, w.climate_data_fullpathname)
#print(output)

## Generate World Template and World File
Now we are almost ready to create the world file for our watershed. First we must create the template from which the world file will be created. To do this, we'll use the **GenerateWorldTemplate** tool. Fortunately this is very easy because the metadata store contains nearly all the information needed to create the template.

The -c (a.k.a. --climateStation) option specifies the climate station to associate with this world file template. The --aspectMinSlopeOne option is necessary work around limitations in the program used to create the world file when the input DEM has areas of low slope.


### Please note: If you used a different resource from HydroShare, you will need to modify the climate station parameter.

In [ ]:
options = " -p " + w.sub_project_folder + " -c cwt"
output = w.GenerateWorldTemplateOptions(options)
print(output)

Now use the **CreateWorldfile** tool to create a world file using this template. <br/>
Note: Output from CreateWorldfile has been piped to output variable. For debugging, uncomment print statement (i.e. remove #).

In [ ]:
output = w.CreateWorldfile(w.sub_project_folder)
#print(output)

## Create Flow Table
As with worldfile creation, at this point in the workflow, RHESSysWorkflows's metadata store contains all the information needed to create a flow table using the createflowpaths (CF) RHESSys program. We'll use CreateFlowtable to create our flow table.

This will result in the creation of a flow table called world.flow in the flow directory of your rhessys directory. Now we have almost everything we need to run RHESSys simulations.

See the RHESSysWorkflows tutorial to learn how to [route surface flows for road pixels directly to the stream](https://github.com/selimnairb/RHESSysWorkflows#creating-the-flow-table) and [create surface flow tables using a roof connectivity map](https://github.com/selimnairb/RHESSysWorkflows#creating-a-surface-flow-table-using-a-roof-connectivity-map)


In [ ]:
options = " -p " + w.sub_project_folder + " --routeRoads"
output = w.CreateFlowtableOptions(options)
print(output)

## Initializing vegetation carbon and nitrogen stores
RHESSys provides a program called LAIread to initialize vegetation carbon and nitrogen stores in your world file.

Initializing carbon and nitrogen stores is a multi-step process that involves running LAI read to generate a redefine worldfile, running a 3-day RHESSys simulation to incorporate the redefine worldfile, writing out a new worldfile with initialized vegetation carbon and nitrogen stores. RHESSysWorkflows automates all of these processes for you using the tool RunLAIRead, which can even figure out what date to start the 3-day RHESSys simulation on based on your climate data.

In [ ]:
output = w.RunLAIRead(w.sub_project_folder)
#print(output)

## Creating a RHESSys TEC file
We need one more thing before we can run our model, a TEC file. TEC stands for "temporal event controller". We use a TEC file to tell RHESSys to do things on at certain times during a simulation.

In [ ]:
output = w.RunCmd(w.sub_project_folder, 3)

## Running RHESSys models
Once you have built a RHESSys model using RHESSysWorkflows, you are free to run your model manually. However this will not capture information about model runs in your project metadata. If you would like to record your runs in your project metadata, use the **RunModel** command.


Because the project metadata knows where RHESSys is installed in your project directory, you don't have to specify the full path of any of the RHESSys input files (e.g. world files, flow tables, TEC files, etc), you only need to specify the filenames. RunModel will echo RHESSys console outlet to the screen (if the -v or verbose option is specified as above), and will always save the same output into a file named 'rhessys.out' stored in the output folder for each model run. The output folder will be named based on the value you provide for the '-pre' or output prefix option. Model output is stored in the output directory of the rhessys directory of your project directory.

For more details on using RunModel see the RHESSysWorkflows [tutorial](https://github.com/selimnairb/RHESSysWorkflows#running-rhessys-models).

### The RunModel function run the RHESSys simulation! Please wait a couple minutes.

In [ ]:
output = w.RunModel(w.sub_project_folder)
print(output)

## Resources

* RHESSys
  * [Setup](http://fiesta.bren.ucsb.edu/~rhessys/setup/setup.html)
  * [Wiki](http://fiesta.bren.ucsb.edu/~rhessys/)
* Data
  * [HydroShare](https://www.hydroshare.org/)
  * [USGS Data and Tools](https://www.usgs.gov/products/data-and-tools/data-and-tools-topics)
  * [USDA Data gateway](https://gdg.sc.egov.usda.gov/)
  * [HydroTerre](http://hydroterre.psu.edu/)
  

## Funding
This work was supported by the following NSF grants

    Award no. 1239678 EAGER: Collaborative Research: Interoperability Testbed-Assessing a Layered Architecture for Integration of Existing Capabilities

    Award no. 0940841 DataNet Federation Consortium.

    Award no. 1148090 Collaborative Research: SI2-SSI: An Interactive Software Infrastructure for Sustaining Collaborative Innovation in the Hydrologic Sciences



## Credits

**Lawrence E. Band** lband@email.unc.edu <br/>
**Brian Miles** brian_miles@unc.edu <br/> 
**Laurence Lin** laurence.lin@icloud.com <br/>
**John Duncan** jon.m.duncan@gmail.com <br/>
**Lorne Leonard** lnl3@psu.edu <br/>
